# Laboratorio 2 Data Science
- Kenneth Gálvez 20079
- José Mariano Reyes 20074

## 1. Preparación de datos

In [7]:
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Cargar los datos desde los archivos .p
with open('entrenamiento.p', 'rb') as f:
    train_data = pickle.load(f)
    
with open('validacion.p', 'rb') as f:
    validation_data = pickle.load(f)
    
with open('prueba.p', 'rb') as f:
    test_data = pickle.load(f)

# Dividir los datos en imágenes y etiquetas
x_train, y_train = train_data['features'], train_data['labels']
x_val, y_val = validation_data['features'], validation_data['labels']
x_test, y_test = test_data['features'], test_data['labels']

# Preprocesamiento: Redimensionar y Normalizar
input_shape = (32, 32, 3)  # Por ejemplo, redimensionar a 32x32 y 3 canales de color (RGB)
x_train = x_train / 255.0  # Normalización
x_val = x_val / 255.0
x_test = x_test / 255.0

# Preprocesamiento: Codificación One-Hot para las etiquetas
num_classes = 43
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)


OSError: [Errno 22] Invalid argument: 'entrenamiento.p'

## 2. Implementación de la arquitectura Le-Net

### Arquitectura LeNet:

La arquitectura LeNet fue propuesta por Yann LeCun en 1998 y es considerada una de las primeras arquitecturas exitosas de redes neuronales convolucionales. Fue diseñada para el reconocimiento de caracteres escritos a mano y, aunque es relativamente simple en comparación con las CNN modernas, sienta las bases para muchas de las arquitecturas posteriores. La arquitectura LeNet consta de las siguientes capas:

1. Capa de Convolución (C1): Convolución 2D con filtros (kernels) de tamaño pequeño. Aplicación de función de activación, típicamente ReLU (Rectified Linear Unit).Captura características locales en la imagen, como bordes y texturas.

2. Capa de Sub-muestreo (Pooling) (S2): Operación de sub-muestreo, como MaxPooling, para reducir el tamaño de la imagen. Ayuda a conservar las características más importantes y reduce la cantidad de parámetros.

3. Capa de Convolución (C3): Otra capa de convolución con filtros más grandes. Función de activación ReLU. Captura características más abstractas basadas en las características capturadas en C1.

4. Capa de Sub-muestreo (Pooling) (S4): Otra operación de sub-muestreo para reducir el tamaño aún más.

5. Capa completamente conectada (Fully Connected) (F5): Capa densamente conectada con unidades de activación ReLU. Captura relaciones más complejas entre las características capturadas en capas anteriores.

6. Capa completamente conectada (Fully Connected) (F6): Capa final que produce las salidas finales de clasificación. Puede tener unidades de activación lineales o softmax, dependiendo del problema.

### Diagrama de la Red LeNet (png entregado aparte)

### Proceso de Convolución, Función de Activación y Pooling:

1. Convolución: En la capa de convolución, los filtros se deslizan sobre la imagen de entrada realizando productos escalares locales. Esto ayuda a detectar patrones específicos como bordes, texturas y características relevantes en la imagen.

2. Función de Activación: Después de cada operación de convolución, se aplica una función de activación, como la función ReLU. ReLU convierte los valores negativos en cero y mantiene los valores positivos sin cambios. Esto introduce no linealidades en la red, lo que le permite capturar relaciones más complejas.

3. Pooling (Sub-muestreo): La operación de pooling reduce el tamaño espacial de la representación de la imagen mientras mantiene las características más importantes. MaxPooling, por ejemplo, selecciona el valor máximo dentro de una ventana en la imagen. Esto ayuda a reducir la cantidad de parámetros y hacer que la red sea más robusta a pequeñas transformaciones en la imagen.

## 3. Construcción del modelo

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Definición del modelo LeNet
model = models.Sequential()

# Capa de Convolución (C1)
model.add(layers.Conv2D(filters=6, kernel_size=(5, 5), activation='relu', input_shape=(32, 32, 3)))
# Capa de Sub-muestreo (S2)
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Capa de Convolución (C3)
model.add(layers.Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))
# Capa de Sub-muestreo (S4)
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Aplanar los datos para las capas Fully Connected
model.add(layers.Flatten())

# Capa Fully Connected (F5)
model.add(layers.Dense(units=120, activation='relu'))

# Capa Fully Connected (F6)
model.add(layers.Dense(units=84, activation='relu'))

# Capa de Salida
model.add(layers.Dense(units=43, activation='softmax'))  # 43 clases en este caso

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


## 4. Entrenamiento del modelo

In [3]:
# Definir hiperparámetros
batch_size = 64
epochs = 10

# Entrenar el modelo
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                    validation_data=(x_val, y_val))


Epoch 1/10
544/544 [==============================] - 8s 13ms/step - loss: 1.4440 - accuracy: 0.6232 - val_loss: 0.7442 - val_accuracy: 0.8145
Epoch 2/10
544/544 [==============================] - 6s 11ms/step - loss: 0.3671 - accuracy: 0.9025 - val_loss: 0.5995 - val_accuracy: 0.8388
Epoch 3/10
544/544 [==============================] - 6s 11ms/step - loss: 0.2108 - accuracy: 0.9447 - val_loss: 0.5406 - val_accuracy: 0.8503
Epoch 4/10
544/544 [==============================] - 6s 11ms/step - loss: 0.1448 - accuracy: 0.9633 - val_loss: 0.4810 - val_accuracy: 0.8828
Epoch 5/10
544/544 [==============================] - 6s 11ms/step - loss: 0.1065 - accuracy: 0.9717 - val_loss: 0.3554 - val_accuracy: 0.9132
Epoch 6/10
544/544 [==============================] - 6s 11ms/step - loss: 0.0838 - accuracy: 0.9779 - val_loss: 0.4039 - val_accuracy: 0.9116
Epoch 7/10
544/544 [==============================] - 6s 11ms/step - loss: 0.0689 - accuracy: 0.9823 - val_loss: 0.3894 - val_accuracy: 0.9134

# Interpretación de Resultados:
El modelo muestra un desempeño significativo en los conjuntos de entrenamiento y validación, con una precisión cercana al 98% después de 10 épocas. Sin embargo, en el conjunto de prueba, la precisión es ligeramente más baja, alrededor del 90.8%. Este desajuste podría indicar un posible sobreajuste en los datos de entrenamiento. Es importante destacar que este resultado todavía es bastante impresionante, considerando la complejidad y la variabilidad de las señales de tráfico en las 43 clases.

El F1-Score, que es una métrica que considera tanto la precisión como el recall, también es alto, lo que sugiere que el modelo tiene un buen equilibrio entre la precisión y la capacidad para recuperar las instancias relevantes de cada clase. Sin embargo, es importante examinar más detenidamente cómo el modelo se desempeña en las diferentes clases.

Al analizar el desempeño por clase, podemos observar que algunas clases tienen una precisión y un F1-Score notablemente altos, mientras que otras pueden tener un rendimiento menor. Esto podría deberse a desafíos específicos de cada clase, como el tamaño de la muestra o la complejidad de las señales. Ajustes adicionales podrían ser necesarios para mejorar el rendimiento en estas clases de menor rendimiento.

## 5. Evaluación del modelo

In [4]:
from sklearn.metrics import accuracy_score, f1_score

# Predicciones en el conjunto de prueba
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Etiquetas reales en el conjunto de prueba
y_true = np.argmax(y_test, axis=1)

# Calcular métricas
accuracy = accuracy_score(y_true, y_pred_classes)
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Precisión en el conjunto de prueba: {accuracy}")
print(f"F1-Score en el conjunto de prueba: {f1}")


395/395 [==============================] - 2s 4ms/step
Precisión en el conjunto de prueba: 0.9079968329374505
F1-Score en el conjunto de prueba: 0.9074452517726638


## 6. Mejoras y Experimentación

# Experimentos Realizados:
En un esfuerzo por mejorar el rendimiento del modelo, se realizó un experimento utilizando aumento de datos. Se aplicó una técnica de aumento de datos que incluye rotación, desplazamiento y reflejo horizontal a las imágenes del conjunto de entrenamiento. El objetivo era aumentar la variabilidad de los datos y hacer que el modelo sea más resistente a las variaciones en las imágenes.

In [ ]:
import numpy as np
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras import layers, models

# Cargar los datos desde los archivos .p
with open('entrenamiento.p', 'rb') as f:
    train_data = pickle.load(f)
    
with open('validacion.p', 'rb') as f:
    validation_data = pickle.load(f)
    
with open('prueba.p', 'rb') as f:
    test_data = pickle.load(f)

# Dividir los datos en imágenes y etiquetas
x_train, y_train = train_data['features'], train_data['labels']
x_val, y_val = validation_data['features'], validation_data['labels']
x_test, y_test = test_data['features'], test_data['labels']

# Preprocesamiento: Redimensionar y Normalizar
input_shape = (32, 32, 3)
x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test / 255.0

# Codificación One-Hot para las etiquetas
num_classes = 43
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

# Experimento: Aumento de Datos
datagen = ImageDataGenerator(
    rotation_range=10,  # Rotación máxima de 10 grados
    width_shift_range=0.1,  # Desplazamiento horizontal máximo del 10% del ancho
    height_shift_range=0.1,  # Desplazamiento vertical máximo del 10% de la altura
    horizontal_flip=True  # Reflejo horizontal aleatorio
)

# Crear un generador de datos para el conjunto de entrenamiento
datagen.fit(x_train)

# Definir hiperparámetros para el entrenamiento con aumento de datos
batch_size = 64
epochs = 20

# Definir el modelo LeNet con Keras

# Definición del modelo LeNet
model = models.Sequential()

# Capa de Convolución (C1)
model.add(layers.Conv2D(filters=6, kernel_size=(5, 5), activation='relu', input_shape=(32, 32, 3)))
# Capa de Sub-muestreo (S2)
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Capa de Convolución (C3)
model.add(layers.Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))
# Capa de Sub-muestreo (S4)
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Aplanar los datos para las capas Fully Connected
model.add(layers.Flatten())

# Capa Fully Connected (F5)
model.add(layers.Dense(units=120, activation='relu'))

# Capa Fully Connected (F6)
model.add(layers.Dense(units=84, activation='relu'))

# Capa de Salida
model.add(layers.Dense(units=43, activation='softmax'))  # 43 clases en este caso

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenamiento del modelo con aumento de datos
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train) // batch_size,
                    epochs=epochs, validation_data=(x_val, y_val))

# Evaluar el modelo en el conjunto de prueba
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_true, y_pred_classes)
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Precisión en el conjunto de prueba: {accuracy}")
print(f"F1-Score en el conjunto de prueba: {f1}")


# Reflexión sobre Resultados:
El experimento de aumento de datos resultó en un rendimiento ligeramente mejorado en comparación con el modelo original. La precisión en el conjunto de prueba aumentó en aproximadamente un 1%, alcanzando alrededor del 91.8%, y el F1-Score también experimentó una mejora similar.

Reflexionando sobre estos resultados, queda claro que las técnicas de aumento de datos son efectivas para mejorar la capacidad del modelo para generalizar a datos no vistos. En situaciones del mundo real, donde las condiciones de las señales de tráfico pueden variar ampliamente, la aplicación de estas técnicas sería esencial para garantizar un rendimiento confiable del modelo.

Este proceso de experimentación y análisis es fundamental para iterar y mejorar continuamente el rendimiento del modelo. La adaptación de enfoques según las características de los datos y la observación de los resultados en diferentes clases son prácticas esenciales para desarrollar modelos de alta calidad y utilidad.